In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
from stratosphere import Stratosphere
from stratosphere.utils.inspect_flows import InspectFlows
from stratosphere import options
from stratosphere.services.extractor import Extractor
from stratosphere.storage.models import Flow

In [3]:
pd.options.display.max_rows = 10
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 100

[[Home](http://127.0.0.1/)]
[[Webapps](http://127.0.0.1/jupyter/voila/tree/webapps)]

# Entity overview



In [4]:
kb = Stratosphere(options.get("db.url_kb"))


In [5]:
import os
from urllib.parse import parse_qs

params = parse_qs(os.environ.get("QUERY_STRING"), keep_blank_values=True)

if 'entity_id' not in params:
    params['entity_id'] = '268acd531d7668dbdb79f33bbfb4575c'


In [6]:
entity_id = params['entity_id'][0]

In [7]:
def get_entity(entity_id):
    
    q = f"""
        SELECT
            *
        FROM entities
        WHERE entity_id = "{entity_id}"
    """
    df = kb.db.pandas(q)
    
    
    return df

get_entity(entity_id)

,entity_id,group,type,data,blob,timestamp


In [8]:
import json
from IPython.display import HTML
from markdown import markdown

def render_entity(entity_id):

    mks= []

    for key, value in get_entity(entity_id).iloc[0].to_dict().items():    
        if key == "blob":
            continue
        if key == 'data':
            mks.append("* **data**")
            data = json.loads(value)
            for data_key, data_value in data.items():
                mks.append(f'    * **{data_key}**: {data_value}')                    
        else:
            mks.append(f'* **{key}**: {value}')

    html = markdown('\n'.join(mks))
    display(HTML(html))

render_entity(entity_id)

IndexError: single positional indexer is out-of-bounds

In [77]:
def get_relationships(entity_id):
    
    q = f"""
        SELECT
            *
        FROM relationships
        WHERE dst = "{entity_id}" or src = "{entity_id}"
    """
    df = kb.db.pandas(q)
    
    return df

df = get_relationships(entity_id)
df.fillna("")


,relationship_id,group,type,src,dst,data,blob,timestamp
0,c2f1f15e619dc3c63378ff396721cfbd,google_search,search_result,7d0677a38f14268564fe031532d3e86b,85b2824c4b33376e91292b9e0eb023e0,"{""url"": ""https://thegrayzone.com/2021/10/09/bellingcat-intelligence-contractors-extremists-syria...",NaN,2023-04-09 15:25:48.032190
